# Content Creator With Reflexion

### Define Pydantic DataModels

In [1]:
from pydantic import BaseModel, Field
from typing import List

In [2]:
class AnswerQuestion(BaseModel):
    """Answer the Question"""
    # thought_process:str = Field(description="Thought process behind the answer. This is between the think tokens in the output of the model. <think>thought_process</think>")
    answer:str = Field(description= "Extremely detailed answer to the question")
    search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer")
    missing: str = Field(description="Critique of what information is missing")
    superfluous: str = Field(description="Critique of what is superfluous")
    
    
class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question"""
    references: List[str] = Field(description="Citations motivating your updated answer")


### Define Chains

In [3]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import PydanticToolsParser, JsonOutputToolsParser
from langchain_core.messages import HumanMessage
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser

In [4]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert content creator who creates interesting, exciting and mind bending content for youtube and instagram.
            Current time: {time}

            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Identify superflous information
            4. After this, **list 1-3 search queries separately** for researching improvements. Do not include them inside the reflection.

            You are required to provide the output in the following json format
            {output_format}
            """
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "."),
    ]
).partial(
    time= lambda: datetime.datetime.now().isoformat(),
)

In [5]:
format_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are helpful assistant who takes the input and structures in the given format
        {input}

        {format_instructions}
        """)
    ]
)

def get_output_format_prompt(pydantic_data_model, format_prompt_template):
    parser = PydanticOutputParser(pydantic_object=pydantic_data_model)
    format_instructions = parser.get_format_instructions()
    format_prompt = format_prompt_template.partial(format_instructions = format_instructions)

    return format_prompt

In [6]:
first_responder_output_instruction = f"""
```json
{AnswerQuestion.model_fields}
```
"""

actor_agent_output_prompt = get_output_format_prompt(AnswerQuestion, format_prompt_template)

In [7]:
revisor_output_instruction = f"""
```json
{ReviseAnswer.model_fields}
```
"""
revisor_agent_output_prompt = get_output_format_prompt(pydantic_data_model=ReviseAnswer, format_prompt_template=format_prompt_template)

In [9]:
# main_llm_model =  "qwen2.5:32b"
# main_llm_model =  "qwen3:32b"
# main_llm_model =  "gpt-oss:20b"
main_llm_model =  "gpt-oss:120b-cloud"
sub_llm_model = "qwen3:1.7b"



llm = ChatOpenAI(
    api_key="ollama",
    model = main_llm_model,
    base_url="http://localhost:11434/v1",
    temperature=1,
)

# using smaller model for output format
output_llm = ChatOpenAI(
    api_key="ollama",
    model=sub_llm_model,
    base_url="http://localhost:11434/v1",
    temperature=0
)



In [10]:
await llm.ainvoke("hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 68, 'total_tokens': 104, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-oss:120b-cloud', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-160', 'finish_reason': 'stop', 'logprobs': None}, id='run--208da689-7adf-42ae-85cb-961048571d16-0', usage_metadata={'input_tokens': 68, 'output_tokens': 36, 'total_tokens': 104, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction = "Provide a extremely detailed answer", 
                                                                output_format = first_responder_output_instruction)

# first_responder_chain = first_responder_prompt_template | llm | actor_agent_output_prompt | output_llm.with_structured_output(schema=AnswerQuestion)
# first_responder_chain = first_responder_prompt_template | llm.with_structured_output(schema=AnswerQuestion)

from langchain_core.output_parsers import JsonOutputParser
first_responder_parser  = JsonOutputParser(pydantic_object = AnswerQuestion)
first_responder_chain = first_responder_prompt_template | llm | first_responder_parser

In [12]:
revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer.
"""

In [13]:
# revisor_chain = actor_prompt_template.partial(first_instruction=revise_instructions,
#                                               output_format = revisor_output_instruction) | llm | revisor_agent_output_prompt | output_llm.with_structured_output(schema=ReviseAnswer)

# revisor_chain = actor_prompt_template.partial(first_instruction=revise_instructions,
#                                               output_format = revisor_output_instruction) | llm.with_structured_output(schema=ReviseAnswer)


from langchain_core.output_parsers import JsonOutputParser
revisor_output_parser  = JsonOutputParser(pydantic_object = ReviseAnswer)
revisor_chain = actor_prompt_template.partial(first_instruction=revise_instructions,
                                              output_format = revisor_output_instruction) | llm | revisor_output_parser

In [14]:
final_content_creator_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """##Personality
            You are an expert content creator who creates interesting, exciting and mind bending content for youtube and instagram from the given content.
            You are given a user's query and answers from different agents and tools as observations which cover different aspects, topics and \
            nuances that are pertaining to the user's query. You are required to create an extremely detailed content from these observations covering all the points \
            which answer's the user question and you need to make sure the generated content is for sure going be a viral and reach Millions. 
            You are required to generate the content for the user query from the given observations only.
            """
        ),
        MessagesPlaceholder(variable_name="user_query"),
        MessagesPlaceholder(variable_name="observations"),
        (
            "system",
            "Generate a VIRAL content for YOUTUBE and INSTAGRAM based on the user_query and observations"
        ),
        MessagesPlaceholder(variable_name="enable_thinking", optional=True)
    ]
)

content_creator_chain = final_content_creator_prompt | llm

In [15]:
summarizer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Create a summary of the messages between the agents and the user.
            The summary must be a short description of the conversation so far, but that also captures all the 
            relevant information shared between the agents and the user: 
            """
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
summarizer_chain = summarizer_prompt | llm

In [49]:
print(summary.content)

<|start|>assistant<|channel|>analysis<|message|>We need to respond to user request: earlier user asked: "dropshipping, e-commerce and its business success in 2025". We responded with summary, missing and superfluous info, etc. Now user maybe wants more? They didn't ask follow up. According to instruction earlier: "Create a summary of the messages between the agents and the user. The summary must be a short description of the conversation so far, but that also captures all the relevant information shared between the agents and the user". We already did. But user didn't ask anything else after providing query. So maybe we need to produce the summary. Wait: In conversation, user first asked about dropshipping. Assistant responded with answer. Then Assistant gave summarization of the conversation? Actually last assistant response was the summary with missing, superfluous, references. That was the answer from previous turn. That was the assistant's final. Now user hasn't responded. They mig

### Testing the chain

In [17]:
input_ = {
    "messages": [HumanMessage("instagram influencers earning more than software developers /no_think")]
}

async for event in first_responder_chain.astream_events(input_):
    if event['event'] == "on_chat_model_stream":
        print(event['data']['chunk'].content, end = "", flush=True)

{
  "answer": "### Overview\nInstagram influencers can, in many cases, out‑earn software developers, especially when they reach a significant follower count and successfully monetize their audience. The discrepancy arises because influencers tap into multiple revenue streams that scale with audience size, while software developer salaries are typically bounded by market rates, experience level, and geographic location.\n\n---\n\n### 1. How Instagram Influencers Make Money\n| Revenue Stream | Description | Typical Earnings Range |\n|----------------|-------------|------------------------|\n| **Sponsored Posts** | Brands pay per post/Story to showcase a product. Rates are usually calculated per 1,000 followers (CPM) but can also be flat fees for macro‑influencers. | $10‑$100 per 1k followers per post (micro‑infl.) → $5k‑$100k+ per post for mega‑infl. |\n| **Affiliate Marketing** | Influencer shares a trackable link or promo code and receives a commission on sales. | 5‑30% commission; top

In [18]:
response =await first_responder_chain.ainvoke({
    "messages": [HumanMessage("instagram influencers earning more than software developers")]
})

for key, val in response.items():
    print("#####" + key + "#####")
    print(val)

#####answer#####
### Overview
Instagram influencers have transformed the platform from a simple photo‑sharing app into a multi‑billion‑dollar advertising ecosystem.  In many cases, top‑tier creators generate revenues that rival or exceed the median salaries of software developers, especially when you factor in bonuses, equity, and regional cost‑of‑living differences.  Below is an exhaustive breakdown that covers:

---

## 1. Quantitative Comparison – Earnings

| Category | Median / Average Annual Income (USD) | Typical Range (USD) | Sources |
|----------|--------------------------------------|---------------------|---------|
| **Software Developer (U.S.)** | $115,000 (Glassdoor 2024) | $80k – $180k (incl. bonuses) | Glassdoor, Stack Overflow Survey 2023 |
| **Software Developer (EU, avg.)** | $70,000 | $50k – $110k | Eurostat, Payscale |
| **Software Developer (India)** | $18,000 | $8k – $35k | Naukri, Indeed |
| **Micro‑Influencer (10k‑100k followers)** | $10,000 – $30,000 | $5k – $40

In [29]:
response =await first_responder_chain.ainvoke({
    "messages": [HumanMessage("AI taking over content creation")]
})

for key, val in response.items():
    print("#####" + key + "#####")
    print(val)

#####answer#####
AI taking over content creation is not a single event but a multifaceted transformation that reshapes every stage of the content lifecycle—from ideation and research to production and distribution. Below is an exhaustive examination of the phenomenon, broken into six interlocking dimensions:

1. **Definition & Scope**
   * **What qualifies as “AI‑driven content?”**  – from natural‑language generation (chat‑GPT‑style models) and auto‑editing video (Auto‑Edit, Rephrase), to algorithm‑optimized thumbnails and audience‑segmentation tools.
   * **Platforms Under the Lens** – YouTube’s Shorts, TikTok’s Reels, Instagram Reels, and emerging VR/AR storytelling spaces. Each has its own AI‑infrastructure (YouTube’s Auto‑Captioning and Content ID, TikTok’s recommendation algorithm). 
   * **Who are the stakeholders?**  – content creators, agencies, platform engineers, algorithmists, advertisers, brand managers, and the audience itself.

2. **The AI Ecosystem in Content Creation**


### Tools

In [19]:
from dotenv import load_dotenv
import os
import json
from typing import List, Dict, Any
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import StructuredTool
from langchain.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [20]:
# Load environment variables from .env file
load_dotenv()

# Access the Tavily API key
tavily_api_key = os.getenv("TAVILY_API_KEY")

if tavily_api_key:
    print("Tavily API Key loaded successfully.")
else:
    print("Tavily API Key not found. Please check your .env file.")

Tavily API Key loaded successfully.


In [21]:
from langchain_core.tools import StructuredTool

from langgraph.prebuilt import ToolNode

tavily_tool = TavilySearchResults(max_results=3)
@tool
def run_queries(search_queries: list[str]):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])

### Graph

In [ ]:
from typing import List, Union
from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph, StateGraph, START
from typing import Literal, Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import ToolCall
import uuid

import time

from loguru import logger

In [26]:
MAX_ITERATIONS = 1

class State(TypedDict):
    user_query: HumanMessage
    observations: List[AIMessage]
    messages: Annotated[list, add_messages]
    revise_answer_iterations: int
    

async def responder_node(state:dict) -> dict:
    logger.info("running responder node")
    logger.info(f"input :: {state['messages']}")
    response = await first_responder_chain.ainvoke({'messages': state['messages']})
    response = AnswerQuestion(**response)
    
    search_tool_call = ToolCall(id = str(uuid.uuid4()),
                                name=response.__class__.__name__, 
                                args = {"queries":response.search_queries})
    
    ai_message = AIMessage(content=str(response.model_dump_json(exclude = 'search_queries', indent=4)), 
                           tool_calls=[search_tool_call])
    
    return {"user_query": state['messages'],
            "observations": [AIMessage(content = "Responder Agent:: " + response.model_dump_json(include = 'answer'))],
            "messages": state['messages'] + [ai_message],
            "revise_answer_iterations": 0}

async def revisor_node(state:dict)-> dict:
    
    logger.info("running revisor node")
    
    response = await revisor_chain.ainvoke({'messages': state['messages']})
    response = ReviseAnswer(**response)
    search_tool_call = ToolCall(id = str(uuid.uuid4()),
                                name=response.__class__.__name__, 
                                args = {"queries":response.search_queries})
    
    ai_message = AIMessage(content=str(response.model_dump_json(exclude = 'search_queries', indent=4)), 
                           tool_calls=[search_tool_call])
    
    return {"user_query": state['user_query'],
            "observations": state["observations"] + [AIMessage(content = "Revisor Agent:: " + response.model_dump_json(include = 'answer'))],
            "messages": state['messages'] + [ai_message],
            "revise_answer_iterations": state['revise_answer_iterations']}

async def execute_tools(state: dict) -> dict:

    logger.info("Executing the tools ")
    messages = state['messages']
    last_ai_message: AIMessage = messages[-1]

    if not hasattr(last_ai_message, "tool_calls") or not last_ai_message.tool_calls:
        return []

    tool_messages = []
    tool_observations = []
    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuestion", "ReviseAnswer"]:
            if tool_call["name"] == 'ReviseAnswer':
                iterations = state['revise_answer_iterations'] + 1
            else:
                iterations = state['revise_answer_iterations']
            
            call_id = tool_call["id"]
            search_queries = tool_call["args"].get("queries", [])
            logger.info(f"Search Queries :: {search_queries}")
            query_results_lst = await run_queries.ainvoke({'search_queries': search_queries})
            query_results = {query:query_result for query,query_result in zip(search_queries, query_results_lst)}
            
            tool_messages.append(
                ToolMessage(content=json.dumps(query_results),
                            tool_call_id = call_id)
            )
            tool_observations.append(json.dumps(query_results))

    tool_observations = '\n'.join(tool_observations)
    return {"user_query": state['user_query'],
            "observations": state["observations"] + [AIMessage(content = "Tool Executor Agent:: " + tool_observations)],
            "messages" : messages + tool_messages, 
            "revise_answer_iterations": iterations,
           }

async def content_creator_node(state:dict)->dict:
    logger.info("Running final content creator node")
    
    user_query = state['user_query']
    observations = state['observations']

    fnl_content = await content_creator_chain.ainvoke({
    'user_query' : user_query,
    'observations': observations,
    'enable_thinking': ['/no_think']
    })

    return {"user_query": state['user_query'],
            "observations": state["observations"],
            "messages" : state['messages'] + [fnl_content],
            "revise_answer_iterations": state["revise_answer_iterations"],
           }
    

async def event_loop(state: dict) -> Literal["content_creator", "execute_tools"]:
    num_iterations = state['revise_answer_iterations']
    if num_iterations >= MAX_ITERATIONS:
        return "content_creator"
    else:
        return "execute_tools"


In [27]:
memory = MemorySaver()

In [28]:
graph = StateGraph(State)

graph.add_node('draft', responder_node)
graph.add_node("execute_tools", execute_tools)
graph.add_node("revisor", revisor_node)
graph.add_node("content_creator", content_creator_node)



graph.add_edge(START, 'draft')
graph.add_edge("draft", 'execute_tools')
# graph.add_edge("draft", END)

graph.add_edge('execute_tools', "revisor")
graph.add_conditional_edges('revisor', event_loop)
graph.add_edge("content_creator", END)
app = graph.compile()


In [29]:

from IPython.display import Image

print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	draft(draft)
	execute_tools(execute_tools)
	revisor(revisor)
	content_creator(content_creator)
	__end__([<p>__end__</p>]):::last
	__start__ --> draft;
	content_creator --> __end__;
	draft --> execute_tools;
	execute_tools --> revisor;
	revisor -.-> content_creator;
	revisor -.-> execute_tools;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



```mermaid
graph TD;
	__start__([<p>__start__</p>]):::first
	draft(draft)
	execute_tools(execute_tools)
	revisor(revisor)
	content_creator(content_creator)
	__end__([<p>__end__</p>]):::last
	__start__ --> draft;
	content_creator --> __end__;
	draft --> execute_tools;
	execute_tools --> revisor;
	revisor -.-> content_creator;
	revisor -.-> execute_tools;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

In [30]:
input_  = {
    "messages": [HumanMessage("comparison of dropshipping, e-commerce and its business success in 2025")]
}

response = await app.ainvoke(input_)

2025-12-08 00:20:03.758 | INFO     | __main__:responder_node:11 - running responder node
2025-12-08 00:20:03.759 | INFO     | __main__:responder_node:12 - input :: [HumanMessage(content='comparison of dropshipping, e-commerce and its business success in 2025', additional_kwargs={}, response_metadata={}, id='77ce9307-94e4-44c3-b173-f03578427e90')]
2025-12-08 00:20:18.827 | INFO     | __main__:execute_tools:48 - Executing the tools 
2025-12-08 00:20:18.828 | INFO     | __main__:execute_tools:66 - Search Queries :: ['2025 dropshipping market share and growth statistics', 'latest e-commerce profit margin benchmarks by business model 2025', 'fast ship dropshipping suppliers US Europe 2025']
2025-12-08 00:20:26.601 | INFO     | __main__:revisor_node:30 - running revisor node
2025-12-08 00:20:36.645 | INFO     | __main__:execute_tools:48 - Executing the tools 
2025-12-08 00:20:36.647 | INFO     | __main__:execute_tools:66 - Search Queries :: ['2025 e‑commerce regulatory changes dropshipping F

In [31]:
print(response['messages'][-1].content)

---

## 🎬 **VIRAL VIDEO SCRIPT – “Dropshipping vs. Traditional E‑Commerce in 2025 – Which Model Will Make YOU MILLIONS?”**  
*(Designed to crush YouTube’s algorithm **and** dominate Instagram Reels/TikTok)*  

| Platform | Length | Why It Works |
|----------|--------|--------------|
| **YouTube (Long‑Form)** | 10‑12 min | Full data, story‑telling, SEO‑friendly. |
| **Instagram Reels / TikTok** | 45‑60 sec | Fast hook, visual punch, “swipe‑up‑to‑learn‑more” CTA. |
| **YouTube Shorts** | 60 sec | Repurposes the Reel for cross‑platform reach. |

---

### 1️⃣  **The 5‑SECOND “HOOK” (Both Formats)**
> **“Dropshipping is a $410 BILLION industry in 2025, but the *real* money? It’s still the 70 % of e‑commerce that stocks its own inventory. Stay tuned—I'll show you which model can turn $5 K into $5 M this year!”**  

- **Visual:** Split‑screen: left side a **speedy delivery drone**, right side a **luxury box with custom branding**.  
- **Audio cue:** Quick *whoosh* → *cash register “ding”* (in

In [35]:
config = {
    "configurable": {
        "thread_id": 10
    }
}

response = await app.ainvoke({
    "messages": [HumanMessage("Hi I'm Sridhar")]
}, config=config
)

response

This is the response of the Responder Node :: 
 content='{\n    "answer": "The answer could have explicitly asked for clarification on budget constraints, prior content experience, and whether the creator has existing footage or needs a full production workflow. Additionally, it could mention platform-specific algorithm insights that are crucial for 2025 content strategy.",\n    "missing": "The answer could have explicitly asked for clarification on budget constraints, prior content experience, and whether the creator has existing footage or needs a full production workflow. Additionally, it could mention platform-specific algorithm insights that are crucial for 2025 content strategy.",\n    "superfluous": "Some sections, such as a detailed list of potential format lengths for TikTok versus Instagram, are not necessary given the current focus on YouTube and Instagram Reels. The answer also repeats the idea of ‘consistent posting’ twice in slightly different wording, which is redundant.

In [33]:

input_ = {
    "messages": [HumanMessage("what are the ways to maintain a healthy lifestyle in bangalore for corporate professionals")]
}
response =  await app.ainvoke(input_)


2025-12-08 00:23:41.154 | INFO     | __main__:responder_node:11 - running responder node
2025-12-08 00:23:41.155 | INFO     | __main__:responder_node:12 - input :: [HumanMessage(content='what are the ways to maintain a healthy lifestyle in bangalore for corporate professionals', additional_kwargs={}, response_metadata={}, id='4eb531f2-ff0b-45a5-83fe-8ea7e16b6b65')]
2025-12-08 00:23:53.150 | INFO     | __main__:execute_tools:48 - Executing the tools 
2025-12-08 00:23:53.152 | INFO     | __main__:execute_tools:66 - Search Queries :: ['Bangalore corporate wellness program best practices 2025', 'Healthy lunch delivery services for office workers Bangalore', 'Morning yoga groups for professionals in Bangalore']
2025-12-08 00:23:57.445 | INFO     | __main__:revisor_node:30 - running revisor node
2025-12-08 00:24:10.560 | INFO     | __main__:execute_tools:48 - Executing the tools 
2025-12-08 00:24:10.561 | INFO     | __main__:execute_tools:66 - Search Queries :: ['Bangalore corporate wellness

In [37]:
print(response['messages'][-1].content)

## 🎥 VIRAL CONTENT BLUEPRINT  
**Topic:** “How to Stay *Fit‑Focused* in Bangalore — A Corporate Pro’s Survival Kit”  

**Goal:** 1‑minute Instagram Reel + 8‑minute YouTube video that *explodes* in shares (✓ relatable pain points, ✓ hyper‑local hacks, ✓ visual eye‑catchers, ✓ strong CTA).  

---  
### 1️⃣  QUICK‑LOOK “THE 30‑SECOND HOOK” (Both Formats)

| Visual | Audio/Voice‑over | Text on‑screen |
|--------|------------------|-----------------|
| Drone‑fly‑over of **Cubbon Park** at sunrise, fast‑cut to a traffic jam on **Outer Ring Road**. | “Bangalore ‑ the garden city that *never* sleeps. You’re stuck in traffic, the inbox is exploding, and your health… is on *hold*.” | “Corporate life = 🚗+💻+⏰ ❓” |
| Snap‑quick selfie‑style of you in a crisp shirt, holding a **protein‑packed tiffin**. | “What if I told you the city *already* gave you everything to stay healthy—if you know the shortcuts?” | “Stay tuned for 7 POWER‑HACKS!” |

🔔 **Hook Ratio:** 2 seconds → curiosity, 3 seconds → probl